In [3]:
#52 라이브러리 호출
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

torch.manual_seed(125)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(125)

In [5]:
#53 데이터 전처리
mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(1.0,))
])

In [6]:
#54 데이터셋 내려받기 및 전처리 적용
from torchvision.datasets import MNIST
download_root = '/content/drive/MyDrive/Colab Notebooks/MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [7]:
#55 데이터셋 메모리로 가져오기
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [8]:
#56 변수값 설정
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

In [9]:
#57GRU 셀 네트워크
class GRUCell(nn.Module):
  def __init__(self,input_size,hidden_size, bias=True):
    super(GRUCell, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.bias = bias
    self.x2h = nn.Linear(input_size, 3*hidden_size, bias=bias) #RNN과 달리 GRU에서는 4를 곱한다
    self.h2h = nn.Linear(input_size, 3*hidden_size, bias=bias) #RNN과 달리 GRU에서는 4를 곱한다
    self.reset_parameters()

  def reset_parameters(self):
    std = 1.0/math.sqrt(self.hidden_size)
    for w in self.parameters():
      w.data.uniform_(-std, std)

  def forward(self, x, hidden):
    x = x.view(-1, x.size(1))

    gate_x = self.x2h(x) #LSTM셀에서는 gates를 x2h+h2h로 정의 했지만 GRU셀에서는 개별적인 상태를 유지
    gate_h = self.h2h(hidden)

    gate_x = gate_x.squeeze()
    gate_h = gate_h.squeeze()

    i_r, i_i, i_n = gate_x.chunk(3,1) #총  세개의 게이트를 위해 세개로 쪼갬
    h_r, h_i, h_n = gate_h.chunk(3,1)

    resetgate = F.sigmoid(i_r+h_r)
    inputgate = F.sigmoid(i_i + h_i)
    newgate = F.tanh(i_n+(resetgate * h_n)) #새로운 게이트는 탄젠트 활성화 함수가 적용된 게이트

    hy = newgate + inputgate*(hidden - newgate)
    return hy

In [21]:
#58 전반적인 네트워크 구조
class GRUModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
    super(GRUModel, self).__init__()
    self.hidden_dim = hidden_dim
    self.layer_dim = layer_dim

    self.gru_cell = GRUCell(input_dim, hidden_dim, layer_dim)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    if torch.cuda.is_available():
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
    else :
      h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

    outs = []
    hn = h0[0,:,:] #LSTM셀에서는 셀 상태에 대해서도 정의했었지만 GRU셀에서는 셀은 사용되지 않음

    for seq in range(x.size(1)):
      hn = self.gru_cell(x[:,seq,:],hn)
      outs.append(hn)

    out =outs[-1].squeeze(0)
    out = self.fc(out)
    return out

In [32]:
#59 옵티마이저와 손실 함수 설정
input_dim = 28
hidden_dim = 28
layer_dim = 1
output_dim = 10

model = GRUModel(input_dim, hidden_dim, layer_dim, output_dim)

if torch.cuda.is_available():
  model.cuda()

criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [34]:
#60 모델학습 및 성능 검증
seq_dim = 28
loss_list = []
iter = 0
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    if torch.cuda.is_available():
      images = Variable(images.view(-1, seq_dim, input_dim).cuda())
      labels = Variable(labels.cuda())
    else :
      images = Variable(images.view(-1, seq_dim, input_dim))
      labels = Variable(labels)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    if torch.cuda.is_available():
      loss.cuda()

    loss.backward()
    optimizer.step()

    loss_list.append(loss.item())
    iter += 1
    if iter % 500 == 0:
      correct = 0
      total = 0
      for images, labels in valid_loader:
        if torch.cuda.is_available():
          images = Variable(images.view(-1, seq_dim, input_dim).cuda())
        else :
          images = Variable(images.view(-1, seq_dim, input_dim))

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)

        if torch.cuda.is_available():
          correct += (predicted.cpu() == labels.cpu()).sum()
        else :
          correct += (predicted == labels).sum()
      accuracy = 100 * correct / total
      print('Iteration: {}. Loss:{}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss:0.8988965749740601. Accuracy: 71.06999969482422
Iteration: 1000. Loss:0.5725591778755188. Accuracy: 85.33000183105469
Iteration: 1500. Loss:0.4086204171180725. Accuracy: 91.27999877929688
Iteration: 2000. Loss:0.38924893736839294. Accuracy: 92.86000061035156
Iteration: 2500. Loss:0.3154514729976654. Accuracy: 94.4800033569336
Iteration: 3000. Loss:0.15545672178268433. Accuracy: 95.30000305175781
Iteration: 3500. Loss:0.23027068376541138. Accuracy: 95.43000030517578
Iteration: 4000. Loss:0.13139575719833374. Accuracy: 95.88999938964844
Iteration: 4500. Loss:0.2347663938999176. Accuracy: 95.83000183105469
Iteration: 5000. Loss:0.08070706576108932. Accuracy: 95.75
Iteration: 5500. Loss:0.16477322578430176. Accuracy: 95.69000244140625
Iteration: 6000. Loss:0.06467495858669281. Accuracy: 96.33999633789062
Iteration: 6500. Loss:0.17888538539409637. Accuracy: 96.75
Iteration: 7000. Loss:0.04801046848297119. Accuracy: 96.69999694824219
Iteration: 7500. Loss:0.1368472129106

In [37]:
#61 테스트 데이터셋을 이용한 모델 예측
def evaluate(model, val_iter):
  corrects, total, total_loss = 0,0,0
  model.eval()
  for images, labels in val_iter:
    if torch.cuda.is_available():
      images = Variable(images.view(-1, seq_dim, input_dim).cuda())
    else :
      images = Variable(images.view(-1, seq_dim, input_dim)).to(device)

    logit = model(images).to(device)
    loss = F.cross_entropy(logit, labels, reduction="sum")
    _, predicted = torch.max(logit.data,1)
    total += labels.size(0)
    total_loss += loss.item()
    corrects += (predicted == labels).sum()

  avg_loss = total_loss / len(val_iter.dataset)
  avg_accuracy = corrects / total
  return avg_loss, avg_accuracy

In [38]:
#62 모델 예측 결과
test_loss, test_acc = evaluate(model, test_loader)
print("Test Loss: %5.2f | Test Accuracy: %5.2f"%(test_loss, test_acc))

Test Loss:  0.10 | Test Accuracy:  0.97
